In [ ]:
!pip install langchain_google_vertexai langgraph langchain_chroma

# Agentic RAG

Let's create a `VectorStore` with some data to illustrate the concept.

For that we will use the file `data.json` that contains some descriptions of clothing items and create a list of `Document` objects with it.

In [2]:
import json
from langchain_core.documents import Document

with open("data.json") as input_file:
    items = json.load(input_file)

documents = [Document(f"{item['name']}: {item['description']}") for item in items]

for document in documents[:3]:
    print("---")
    print(document.page_content)
    print("---")

---
Sunny Day Tee: This vibrant yellow t-shirt is perfect for sunny summer days. Made from soft, breathable cotton, it features a classic crew neck and relaxed fit. The cheerful color will brighten up any outfit.
---
---
Denim Dream Pants: These classic blue denim pants are a versatile addition to your fall wardrobe. With a straight leg and mid-rise waist, they offer a comfortable and flattering fit. The durable denim fabric ensures they'll last through many seasons.
---
---
Midnight Magic Coat: Stay warm and stylish in this elegant black winter coat. Crafted from a luxurious wool blend, it features a sleek silhouette and cozy collar. The timeless design makes it a perfect choice for any occasion.
---


Now, for illustration porpouses, let's create an in memory `VectorStore` using `Chroma`. Remember that in `Chapter 4` we explored all Vectorstores available in Google Cloud for production applications that follow the same interface.

In [3]:
from langchain_chroma import Chroma
from langchain_google_vertexai import VertexAIEmbeddings
from langchain.tools.retriever import create_retriever_tool


embedding_model = VertexAIEmbeddings(model_name="textembedding-gecko@003")
vector_store = Chroma.from_documents(documents=documents, embedding=embedding_model)

From the `VectorStore`, we create a `Tool` that is able to be called by an agent.

In [4]:

retriever = vector_store.as_retriever(k = 3)

retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="retrieve_items",
    description="Retrieves clothing item descriptions from the knowledge base."
)

response = retriever_tool.invoke("I want a black coat for winter")
print(response)

Midnight Magic Coat: Stay warm and stylish in this elegant black winter coat. Crafted from a luxurious wool blend, it features a sleek silhouette and cozy collar. The timeless design makes it a perfect choice for any occasion.

Winter Wonderland Jacket: Stay warm and cozy in this winter white jacket. Crafted from a water-resistant fabric, it features a plush lining and faux fur trim. The quilted design adds a touch of style while providing extra insulation against the cold.

Cozy Cabin Cardigan: Snuggle up by the fire in this cozy brown cardigan. Knitted from a soft wool blend, it features a chunky knit pattern and oversized fit. The warm neutral color complements a variety of winter outfits.

Snowy Peaks Parka: Brave the elements in style with this blue parka. Crafted from a water-resistant and windproof fabric, it features a warm down filling and faux fur-lined hood. The multiple pockets and adjustable cuffs provide added functionality.


Let's now create the agent architecture as we described in the Chapter

In [5]:
from typing import Literal
from langchain_google_vertexai import ChatVertexAI
from langchain_core.messages import AnyMessage
from langgraph.graph import StateGraph, MessagesState, END
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, SystemMessage

graph = StateGraph(MessagesState)
tools = [retriever_tool]

generator = ChatVertexAI(
  model_name="gemini-1.5-pro-001", 
  temperature = 0
).bind(tools=tools)

def invoke_generator(state: MessagesState) -> None:
    """ Represents the generator node.
    """
    response = generator.invoke(state["messages"])
    state["messages"].append(response)
    
def use_retrieval(state: MessagesState) -> Literal["retriever_tool", END]:
    """ Represents the use_retrieval conditional edge
    """
    if not state["messages"]:
        return END
    if state["messages"][-1].tool_calls:
        return "retriever_tool"
    return END


graph.add_node("generator", invoke_generator)
graph.add_node("retriever_tool", ToolNode(tools))
graph.set_entry_point("generator")

graph.add_conditional_edges("generator", use_retrieval)
graph.add_edge("retriever_tool", "generator")


agentic_rag = graph.compile()

We can invoke the agent with the `invoke` or `stream` methods. Let's try with a question that doesn't require a function invocation.

In [6]:

messages = [
    SystemMessage(
        """
        - You are a useful assistant that help users navigate a catalog of clothing items.
        - You can retrieve clothing items from the catalog using a natural language query.
        - Answer in natural language and format your output using paragraph or bullet points if
          necessary.
        """
    ),
    HumanMessage("Hello! How can you help me?")
]

stream_generator = agentic_rag.stream({"messages": messages}, stream_mode="values")

for state in stream_generator:
    state["messages"][-1].pretty_print()
state["messages"][-1].pretty_print()

================================ Human Message =================================

Hello! How can you help me?
================================== Ai Message ==================================

I can help you navigate a catalog of clothing items. For example, you can ask me to find items similar to "a red dress" or "a pair of blue jeans". 

What are you looking for today?


If we invoke it with a question, it will automatically call the retriever tool:

In [7]:

messages = [
    SystemMessage(
        """
        - You are a useful assistant that help users navigate a catalog of clothing items.
        - You can retrieve clothing items from the catalog using a natural language query.
        - Answer in natural language and format your output using paragraph or bullet points if
          necessary.
        """
    ),
    HumanMessage("Do you have any yellow t-shirts?")
]

stream_generator = agentic_rag.stream({"messages": messages}, stream_mode="values")

for state in stream_generator:
    state["messages"][-1].pretty_print()
state["messages"][-1].pretty_print()

================================ Human Message =================================

Do you have any yellow t-shirts?
================================= Tool Message =================================
Name: retrieve_items

Sunny Day Tee: This vibrant yellow t-shirt is perfect for sunny summer days. Made from soft, breathable cotton, it features a classic crew neck and relaxed fit. The cheerful color will brighten up any outfit.

Summer Solstice Tank: Stay cool and comfortable in this classic white tank top. Made from a soft cotton jersey, it features a relaxed fit and scoop neck. The versatile design can be dressed up or down for any occasion.

Blossom Blouse: Embrace the beauty of spring with this delicate pink t-shirt. Adorned with a subtle floral print, it exudes femininity and charm. The lightweight fabric and flowy design make it a comfortable and stylish choice.

Nautical Vibes Top: Channel your inner sailor with this nautical-inspired blue and white striped t-shirt. Made from a light